In [33]:
# Retrieving image metadata, including capture time, geolocation information (latitude and longitude),
import exifread
import re
def Time_GSP(path):
    f = open(path,"rb")
    tags = exifread.process_file(f)
    GPS = {}
    Data = ""
    for tag,value in tags.items():
        # Retrieving latitude information
        if re.match('GPS GPSLatitude', tag):
            try:
                match_result=re.match('\[(\w*), (\w*), (\w.*)/(\w.*)\]', str(value)).groups()
                GPS['w'] = str(int(match_result[0])) + " " + str(int(match_result[1])) + " " + str(int(match_result[2])/int(match_result[3]))
            except:
                GPS['w'] = str(value)
        # Retrieving longitude information
        elif re.match('GPS GPSLongitude', tag):
            try:
                match_result=re.match('\[(\w*), (\w*), (\w.*)/(\w.*)\]',str(value)).groups()
                GPS['j'] = str(int(match_result[0])) + " " + str(int(match_result[1])) + " " + str(int(match_result[2])/int(match_result[3]))
            except:
                GPS['j'] = str(value)
        elif re.match('GPS GPSAltitude', tag):
            GPS['h'] = str(value)
        # Timestamp of image capture
        elif re.match('Image DateTime', tag):
            Data = str(value)
    return Data

In [45]:
# model loading 
import tensorflow as tf
from skimage import io
from skimage.transform import resize
import numpy as np
import os
import datetime


save_path = r"./GSP_AI" # 模型的名称
model_loaded = tf.keras.models.load_model(save_path)

# Combine image data with the corresponding timestamp
def tp(img_path):
    img = io.imread(img_path)
    img = resize(img,(224,224,3))
    img_time = Time_GSP(img_path)
    img_time = datetime.datetime(int(img_time[:4]),int(img_time[5:7]),int(img_time[8:10]))
    return  img,img_time

a,img_time = tp(path)
print(img_time)
df_e = pd.read_excel(path_e)
df_new = df_e[df_e['datetime']==img_time]
data_info = df_new.loc[:,'nor_rain':'nor_solar']
print(data_info)

path_img = r'./Cloud_data' # Image data
path_e =  r"./Environment.xlsx" # Environment data
df_e = pd.read_excel(path_e)

images_o = []
info_list = []
for name in os.listdir(path_img):
    img_path = os.path.join(path_img,name)
    new_img,img_time = tp(img_path)
    images_o.append(new_img)
    df_new = df_e[df_e['datetime']==img_time]
    data_info = df_new.loc[:,'nor_rain':'nor_solar']
    info_list.append(data_info)

print(info_list[2])
# Growth stages 
Class_label = ["Tillering", "Jointing", "B$H",'Flowering','Filling','Ripening']
GSP_1 = [];GSP_2 =[];GSP_3 = [];GSP_4=[];GSP_5 =[];GSP_6=[]
img_arr=np.asarray(images_o)

for i in range(len(images_o)-1):
    one_img = np.expand_dims(img_arr[i],axis=0)
    one_info = info_list[i]
    #'label' corresponds to the type of growth stage, and 'FS' corresponds to the flowering time
    label,FS = model_loaded.predict([one_info,one_img])
    GSP_1.append(label[0][0])
    GSP_2.append(label[0][1])
    GSP_3.append(label[0][2])
    GSP_4.append(label[0][3])
    GSP_5.append(label[0][4])
    GSP_6.append(label[0][5])
    pre_value = label[0].tolist()
    k_value = pre_value.index(max(pre_value))+1;
    print('class_num--FS_time:  ',Class_label[k_value-1],format(FS[0][0],'.1f'))


2021-02-20 00:00:00
     nor_rain  nor_agdd  nor_solar
106       0.0     0.223      0.366
     nor_rain  nor_agdd  nor_solar
106       0.0     0.223      0.366
class_num--FS_time:   Tillering 162.2
class_num--FS_time:   Tillering 162.2
class_num--FS_time:   Tillering 161.8
class_num--FS_time:   Tillering 161.9
class_num--FS_time:   Tillering 161.5
class_num--FS_time:   Tillering 161.7
class_num--FS_time:   Tillering 161.9
class_num--FS_time:   Tillering 162.0
class_num--FS_time:   Tillering 161.6
class_num--FS_time:   Tillering 161.6
class_num--FS_time:   Tillering 161.6
class_num--FS_time:   Tillering 161.4
class_num--FS_time:   Tillering 161.9
class_num--FS_time:   Tillering 161.6
class_num--FS_time:   Tillering 161.7
class_num--FS_time:   Tillering 161.5
class_num--FS_time:   Tillering 161.6
class_num--FS_time:   Tillering 161.6
class_num--FS_time:   Jointing 161.4
class_num--FS_time:   Tillering 161.5
class_num--FS_time:   Jointing 161.4
class_num--FS_time:   Jointing 161.9
class_n

class_num--FS_time:   B$H 163.0
class_num--FS_time:   B$H 163.0
class_num--FS_time:   B$H 162.8
class_num--FS_time:   B$H 162.7
class_num--FS_time:   Flowering 162.6
class_num--FS_time:   B$H 162.1
class_num--FS_time:   Flowering 162.8
class_num--FS_time:   B$H 162.9
class_num--FS_time:   B$H 162.7
class_num--FS_time:   B$H 162.9
class_num--FS_time:   B$H 162.8
class_num--FS_time:   B$H 162.8
class_num--FS_time:   B$H 162.6
class_num--FS_time:   B$H 163.3
class_num--FS_time:   B$H 163.0
class_num--FS_time:   B$H 163.3
class_num--FS_time:   B$H 162.9
class_num--FS_time:   B$H 163.0
class_num--FS_time:   Flowering 162.7
class_num--FS_time:   B$H 162.8
class_num--FS_time:   B$H 162.7
class_num--FS_time:   B$H 162.9
class_num--FS_time:   B$H 162.8
class_num--FS_time:   B$H 163.1
class_num--FS_time:   Flowering 162.6
class_num--FS_time:   Flowering 162.7
class_num--FS_time:   Flowering 162.6
class_num--FS_time:   Flowering 162.6
class_num--FS_time:   B$H 162.8
class_num--FS_time:   B$H 162.

class_num--FS_time:   Filling 162.4
class_num--FS_time:   Filling 162.7
class_num--FS_time:   Filling 162.7
class_num--FS_time:   Filling 162.4
class_num--FS_time:   Filling 162.6
class_num--FS_time:   Jointing 162.3
class_num--FS_time:   Jointing 162.4
class_num--FS_time:   Filling 162.4
class_num--FS_time:   Filling 162.3
class_num--FS_time:   Filling 162.2
class_num--FS_time:   Filling 162.5
class_num--FS_time:   Filling 162.3
class_num--FS_time:   Filling 162.2
class_num--FS_time:   Filling 162.5
class_num--FS_time:   Filling 162.7
class_num--FS_time:   Filling 162.7
class_num--FS_time:   Jointing 161.2
class_num--FS_time:   Jointing 161.2
class_num--FS_time:   Jointing 161.2
class_num--FS_time:   Jointing 160.6
class_num--FS_time:   Jointing 160.6
class_num--FS_time:   Ripening 161.0
class_num--FS_time:   Jointing 160.9
class_num--FS_time:   Jointing 161.1
class_num--FS_time:   Ripening 161.0
class_num--FS_time:   Ripening 161.2
class_num--FS_time:   Ripening 161.0
class_num--FS_t